<h1 align="center" style="margin-bottom: 20px;">Business Cases with Data Science 2024-25</h1>
<h3 align="center" style="margin-top: 20px; margin-bottom: 20px;">Case 4: AI - Powered Chatbot</h3>
<h5 align="center" style="margin-top: 20px; margin-bottom: 0px;">Notebook 1 

### Group B - Members:
- Ana Marta Azinheira | 20240496@novaims.unl.pt
- Bráulio Damba | 20240007@novaims.unl.pt
- Jan-Louis Schneider | 20240506@novaims.unl.pt
- Sofia Jacinto | 20240598@novaims.unl.pt

# Notebook Objective

- This notebook is designed to test our AI-Powered Chatbot. Throughout the notebook, we develop and refine various prompts to interact with our model. To support this notebook, we include a separate .py file containing the core functions used by the chatbot. At the end of the notebook, users will be able to test the chatbot in a simulated environment.

# Importing Libraries and Others

In [1]:
# Imports
import os
import json
import time
import pandas as pd
from openai import AzureOpenAI
from PIL import Image
from IPython.display import Markdown, display
import pickle
from datetime import datetime, timedelta
import fitz
from docx import Document  

#pip install pymupdf python-docx
#!pip install python-docx

In [2]:
# Import methods from utils.py
from utils import (
    create_assistant,
    create_thread,
    check_assistant_exists,
    load_and_upload_files,
    add_message_to_thread,
    display_messages,
    send_message_to_assistant
)

In [ ]:
#AQUI VCS PRECISAM COLOCAR O KEY DE VCS. ENTRAM NO PDF 7-LargeLanguageModels  no moodle do curso, a partir da pagina 56 tá explicado. é bem facil
# dps de entrar no azure open ai lá e tipo terem feito o registro, só precisam ir para o chat/playground e dps clicar em 'view code' e dps tiram de lá o 
# key de vcs, que esta lá ocultado e colocam esse key aqui em baixo. o endpoint já tá cert e deve ficar assim

# Set API key and endpoint
api_key = 'yourAPIKey'
endpoint = 'https://ai-bcds.openai.azure.com/'

In [4]:
# Constants
current_folder = os.getcwd()
data_folder = current_folder
data_folder_full_path = os.path.abspath(data_folder)

assistantFilename = 'AssistantID.TXT'
assistant_id = None
assistant = None
vector_data = 'vector_store.pkl'   # Where uploaded data will be/is saved

displayedMessagesIDs = []

In [5]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = endpoint,
    api_key= api_key,
    api_version="2024-05-01-preview")

In [6]:
# Load the link_map created, that links every document with a source link
with open("document_links.json", "r", encoding="utf-8") as f:
    doc_link_map = json.load(f)

In [7]:
# se já existe o vector_store.pkl e nao mudaram nada, isse vai ser bué rapido. IMPORTANTE, SE MEXEM ALGO NOS DOCUMENTOS, TEM QUE
# APAGAR O vector_store.pkl lá na pasta, E CORREM ISSO AQUI, ai vai demorar tipo 15 min pq ele carrega todos os documentos dnv, mas é isso

# Load or upload documents 
if os.path.exists(vector_data):
    with open(vector_data, "rb") as file:
        vector_store = pickle.load(file)
    print("Loaded existing vector store from file.")
else:
    print("Uploading documents and creating vector store...")
    vector_store = load_and_upload_files(client, link_map=doc_link_map)
    print("Documents uploaded.")

Loaded existing vector store from file.


# Rules/Prompts for the Model

In [8]:
# Role of the model
aRole = (
    "És um assistente virtual fiável e rápido que apoia os colaboradores da seguradora Fidelidade.\n"
    "O teu objetivo é fornecer respostas claras e corretas durante os atendimentos a clientes, "
    "especialmente quando o colaborador precisa de consultar informações rapidamente.\n"
    "Não interages diretamente com o cliente final, mas ajudas o colaborador da Fidelidade a responder com confiança."
)

# Some rules/prompts for the model
prompt_rules = (
    "Rules:\n"
    "Responda somente se a resposta puder ser encontrada claramente nos documentos fornecidos.\n"
    "Se não tiver certeza, diga: \"Desculpe, não tenho informação suficiente para responder com precisão.\" Não invente.\n"
    "Se a mensagem for apenas uma saudação, responda de forma rápida e direcione o colaborador para fazer uma pergunta específica.\n"
    "\n"
    "Use uma linguagem clara, direta e profissional, adequada para que o colaborador possa repassar ao cliente.\n"
    "Seja direto, útil e evite respostas muito longas. Quando possível, responda com listas curtas (bullet points).\n"
    "\n"
    "- Sempre que uma resposta for baseada em um documento, verifique se no final do texto existe uma linha que começa com 'Fonte:'.\n"
    "- Se existir, adicione esse link como fonte no final da resposta com a frase: 'Fonte: [link]'.\n"
    "- Se não houver fonte clara, responda normalmente.\n"
    "- Nunca invente links. Use apenas os que já estão presentes no texto fornecido.\n"
    "\n"
    "IMPORTANTE: Cada documento inclui várias vezes no início o campo \"Categoria:\", que indica se o conteúdo é da Fidelidade ou de um concorrente.\n"
    "\n"
    "• Se Categoria for **Fidelidade**, pode usar livremente o conteúdo para responder sobre produtos da Fidelidade.\n"
    "• Se Categoria for **Concorrente**, use essas informações **apenas** para comparações quando for solicitado pelo usuário.\n"
    "  NUNCA use informações de concorrentes para explicar, descrever, recomendar ou apresentar produtos da Fidelidade.\n"
    "\n"
    "Regra de ouro: **Nunca utilize conteúdo de documentos com 'Categoria: Concorrente' para responder diretamente a perguntas sobre produtos da Fidelidade.**\n"
    "\n"
    "Instruções adicionais:\n"
    "– Se a pergunta estiver presente nos Q&As dos documentos, use a resposta exatamente como está lá.\n"
    "– Se a pergunta não estiver nos documentos, mas for básica e relacionada ao tema (ex: conceitos de seguros), pode responder de forma genérica.\n"
    "– Se a pergunta não tiver relação com o tema, responda: \"Essa pergunta não se enquadra no suporte oferecido por este assistente. Consulte um responsável autorizado.\"\n"
    "– Se usar informações de documentos, **forneça sempre o link da página correspondente** como referência.\n"
    "– Se não souber a resposta, mas a pergunta for relevante, indique o site oficial da Fidelidade como fonte adicional.\n"
    "– Para valores que mudam com o tempo (ex: rentabilidades, taxas, prazos), não afirme números fixos a menos que estejam claramente presentes no documento com fonte.\n"
    "– Quando responder a comparações entre produtos, use estrutura neutra com listas ou tabelas curtas, sem emitir opinião.\n"
    "– Se a pergunta for ambígua (ex: \"Qual é o melhor?\"), peça mais contexto antes de responder.\n"
    "– Para dúvidas complexas ou fora do escopo, oriente o colaborador a encaminhar para um especialista humano.\n"
    "– Quando não for possível responder totalmente, pode encerrar com: \"Para mais detalhes, consulte o site oficial da Fidelidade ou os documentos fornecidos.\"\n"
    "\n"
    "# ORIENTAÇÃO COMERCIAL ADICIONAL:\n"
    "- Sempre que possível, destaque os benefícios do produto My Savings de forma clara e natural.\n"
    "- Use uma linguagem que evidencie vantagens para o cliente (ex: segurança, flexibilidade, poupança fiscal, ausência de comissões, simplicidade digital).\n"
    "- Quando responder sobre produtos da Fidelidade, inclua elementos como:\n"
    "  • Gestão profissional\n"
    "  • Solidez da marca Fidelidade\n"
    "  • Produtos pensados para o cliente final\n"
    "- Se fizer sentido, use frases introdutórias comerciais como:\n"
    "  • “O My Savings é ideal para quem valoriza…”\n"
    "  • “Uma solução simples, segura e com benefícios fiscais.”\n"
    "  • “Pensado para o cliente que procura tranquilidade no futuro.”\n"
    "- Sempre que relevante, organize a resposta em bullets destacando os benefícios principais.\n"
    "- Nunca invente benefícios que não estão nos documentos. Apenas realce o que já está lá, com linguagem orientada ao cliente.\n"
    "\n"
    "# EXEMPLO DE FORMATO DE RESPOSTA:\n"
    "O My Savings é uma solução de poupança com perfil conservador, ideal para quem valoriza segurança e tranquilidade:\n"
    "• Capital garantido em determinadas condições\n"
    "• Vantagens fiscais associadas ao regime PPR\n"
    "• Sem comissões de subscrição ou resgate\n"
    "• Gestão profissional pela Fidelidade\n"
    "Fonte: [link]\n"
)

In [9]:
# Load or create assistant using existing vector store

# Try load existing assistant
if os.path.exists(assistantFilename):
    with open(assistantFilename, "r") as file:
        assistant_id = file.read().strip()

    # Check if assistant exists in azure
    exists, assistant = check_assistant_exists(client, assistant_id)
    if exists:
        # Load assistant with new role
        assistant = client.beta.assistants.update(
            assistant_id=assistant_id,
            instructions=aRole, 
            tool_resources={    # The documents made available for the model
                "file_search": {
                    "vector_store_ids": [vector_store.id]
                }
            }
        )
        print("Using existing assistant:", assistant_id)

    else:  # If assistant exist but not valid in azure
        print("Assistant ID found, but not valid in API. Creating new one...")
        assistant_id = None   # Marker to create new assistant 

else:   # If no assistant found
    print("ℹNo assistant ID file found. Creating new assistant...")
    assistant_id = None   # Marker to create new assistant

# No valid assistant --> create new one
if assistant_id is None:
    print("Creating new assistant...")
    assistant = create_assistant(client, aRole, assistantFilename)
    assistant_id = assistant.id

    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={
            "file_search": {
                "vector_store_ids": [vector_store.id]
            }
        }
    )
    print("New assistant created and linked.")

Using existing assistant: asst_afxDUFnwot8aRxGIbDcmPyiv


In [10]:
# New thread
thread = create_thread(client)

# Testing the Chat Bot

In [ ]:
# WRITE 'Obrigado, até à próxima!' TO TERMINATE CHAT or WRITE 'Nova Conversa' to reset chat

while True:
    user_input = input("You: ")
    full_prompt = f"{prompt_rules}\n\nUser question: {user_input}"

    if user_input.strip().upper() == "Obrigado, até à próxima!":
        print("Obrigado pelo seu contacto. Sempre que precisar estarei aqui. Para que a vida não pare.")
        break
    elif user_input.strip().upper() == "Nova Conversa":
        print("Starting a new chat...")
        thread = create_thread(client)
    else:
        send_message_to_assistant(client, thread, assistant, user_input, full_prompt, displayedMessagesIDs)